In [12]:
import xml.etree.ElementTree as ET

def parseXML(xml_path):
    tree = ET.parse(xml_path)
    
    root = tree.getroot()
    for child in root:
        print(child.tag)
        print('=======')
        print(child.attrib)
        print('=======')
        print(child.text)

In [13]:
parseXML('/Users/yinnnyou/workspace/code_vulnerability_tools/srcxml/juliet.java.0.example.xml')

{http://www.srcML.org/srcML/src}package
{}

        package
        
{http://www.srcML.org/srcML/src}import
{}

        import
        
{http://www.srcML.org/srcML/src}import
{}

        import
        
{http://www.srcML.org/srcML/src}import
{}

        import
        
{http://www.srcML.org/srcML/src}import
{}

        import
        
{http://www.srcML.org/srcML/src}class
{}

        
